# Importing some packages

In [1]:
import numpy as np
from numba import njit
import pandas as pd
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

# Limited testing

In [2]:
@njit(cache=True)
def get_signals(signal_list, exit_list):

    start_idx = 0
    exit_idx = 0
   
    for i in range(len(signal_list[0])):

        if i == start_idx:

            if signal_list[0][i] == 0:

                start_idx += 1

                exit_list[0][i] = 0

            else:

                for j in range(i+1, len(exit_list[0])):
                    if exit_list[0][j] == -signal_list[0][i]:
                        exit_idx = j
                        break
                    else:
                        exit_idx = j

                for k in range(i+1, exit_idx+1):
                    if signal_list[0][k] == -signal_list[0][i]:
                        exit_idx = k
                        exit_list[1][k] = signal_list[1][k]
                        break
                    else:
                        exit_idx = k

                for p in range(i+1, exit_idx):
                    signal_list[0][p] = 0
                    exit_list[0][p] = 0
                
                if exit_idx == len(signal_list[0]) - 1 and exit_list[0][exit_idx] != -signal_list[0][i]:
                    exit_list[0][exit_idx] = 0
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                else:
                    exit_list[0][exit_idx] = -signal_list[0][i]
                    exit_list[0][i] = 0
                    signal_list[0][exit_idx] = 0
                
                start_idx = exit_idx + 1

        else:

            continue

    if sum(np.abs(signal_list[0])) == sum(np.abs(exit_list[0])):

        return signal_list, exit_list
    
    else:

        for i in range(len(signal_list[0])):
            if signal_list[0][-(i+1)] != 0:
                signal_list[0][-(i+1)] = 0
                break

        return signal_list, exit_list
    
@njit(cache=True)
def create_position_open_prices(signal_list, exit_list):

    pos_open_prices = np.zeros(len(signal_list[0]))
    pos_exit_prices = np.zeros(len(exit_list[0]))

    start_idx = 0
    price_idx = 0

    for i in range(len(signal_list[0])):
        if exit_list[0][i] != 0:
            for j in range(start_idx, i):
                if signal_list[0][j] != 0:
                    price_idx = j
                    break
            pos_open_prices[i] = signal_list[1][price_idx]
            pos_exit_prices[i] = exit_list[1][i]
            start_idx = i
        else:
            pass

    return pos_open_prices, pos_exit_prices

@njit(cache=True)
def get_pnl_testing(
    trade_close_prices,
    signal_list, 
    trade_open_prices,
    commission=0.015,
    slippage=0.05,
    init_inv=20000,
    trade_size=0.1
):

    pnl_list = np.zeros(len(trade_close_prices))

    for i in range(len(trade_close_prices)):

        if signal_list[i] == 0 or trade_open_prices[i] == 0:
            pass
        
        # signal_list contains the points where exit occurs
        elif signal_list[i] == -1: 
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage)) 
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        else:
            temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
            temp_pnl = temp_n_assets * (trade_open_prices[i] * (1 - slippage) - trade_close_prices[i])
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

        pnl_list[i] = temp_pnl

    return pnl_list


## Entry testing

### Fixed stop and target exit

In [3]:
@njit(cache=True)
def get_exit_entry_testing1(
    close_prices, 
    open_prices,  
    signal_list,
    stoploss_th,
    takeprofit_th, 
    commission, 
    slippage, 
    init_inv, 
    trade_size
):

    exit_list = np.zeros((2, len(close_prices)))

    for i in range(len(close_prices)-1):

        if signal_list[1][i] == 0:

            pass

        else:

            temp_n_assets = int(init_inv * trade_size / signal_list[1][i])
            if signal_list[0][i] == 1:
                temp_pnl = temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 + slippage))
            else:
                temp_pnl = -temp_n_assets * (close_prices[i] - signal_list[1][i] * (1 - slippage))
            temp_pnl = temp_pnl * (1 - commission)
            init_inv += temp_pnl

            if -temp_pnl >= stoploss_th or temp_pnl >= takeprofit_th:
                exit_list[0][i+1] = -signal_list[0][i]
                exit_list[1][i+1] = open_prices[i+1]
            else:
                pass
        
    return exit_list

### Fixed bar exit

In [4]:
@njit(cache=True)
def get_exit_entry_testing2( 
    open_prices,  
    signal_list,
    n_exit_bars
):

    exit_list = np.zeros((2, len(signal_list[0])))

    n_exit_bars = np.int64(n_exit_bars)

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:
            
            if i + n_exit_bars < len(signal_list[0]):
                exit_list[0][i+n_exit_bars] = -signal_list[0][i]
                exit_list[1][i+n_exit_bars] = open_prices[i+n_exit_bars]
            else:
                pass
        
    return exit_list


### Random exit

In [5]:
@njit(cache=True)
def get_exit_entry_testing3( 
    open_prices,  
    signal_list
):

    exit_list = np.zeros((2, len(signal_list[0])))

    for i in range(len(signal_list[0])-1):

        if signal_list[0][i] == 0:

            pass

        else:

            for j in range(i+1, len(signal_list[0])):
                if signal_list[0][j] != 0:
                    j = j - 1
                    break
                else:
                    if np.random.rand() > 0.5:
                        break
            
            exit_list[0][j] = -signal_list[0][i]
            exit_list[1][j] = open_prices[j]
        
    return exit_list


### Winning percentage

In [6]:
def calculate_mean_win_perc_entry_testing(data, text_code):

    bars_per_5week = 7 * 60 * 24 * 5
    n_bars_per_year = 7 * 60 * 24 * 52

    n_not_worked = 0
    n_total_cases = 0

    entry_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = data.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)
        
        try:
            exec_dict = {'data': df}
            exec(text_code, exec_dict)
            df = exec_dict['df']

            commission = exec_dict['COMMISSION']
            slippage = exec_dict['SLIPPAGE'] 
            init_inv = exec_dict['AVAILABLE_CAPITAL']
            trade_size = exec_dict['TRADE_SIZE'] 

            signal_idxs = exec_dict['buy_idxs'].copy()
            signal_idxs.extend(exec_dict['sell_idxs'])
            signal_idxs = sorted(signal_idxs)
            signal_idxs_true = [i - 1 for i in signal_idxs]

            df['new_signal'] = 0
            df.loc[df.index.isin(signal_idxs_true), 'new_signal'] = df.loc[df.index.isin(signal_idxs_true), 'signal'].values
            df['signal_prices'] = 0
            df.loc[df.index.isin(signal_idxs), 'signal_prices'] = df.loc[df.index.isin(signal_idxs), 'open'].values

            signal_list = np.zeros((2, df.shape[0]))
            signal_list[0][1:] = df['new_signal'].values[:-1]
            signal_list[1] = df['signal_prices'].values

            # fixed stop and target exit testing
            exit_list = get_exit_entry_testing1(
                close_prices=df['close'].values, 
                open_prices=df['open'].values,  
                signal_list=signal_list,
                stoploss_th=50,
                takeprofit_th=100,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            entry_walk_forward_dict['fixed_sp_testing'].append(fixed_winning_percent)

            # fixed bar exit testing
            exit_list = get_exit_entry_testing2( 
                open_prices=df['open'].values,  
                signal_list=signal_list,
                n_exit_bars=5
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            entry_walk_forward_dict['fixed_bar_testing'].append(fixed_bar_winning_percent)

            # random exit testing
            exit_list = get_exit_entry_testing3( 
                open_prices=df['open'].values,  
                signal_list=signal_list
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            entry_walk_forward_dict['random_exit_testing'].append(random_winning_percent)

        except:

            n_not_worked += 1

    mean_win_perc_dict = defaultdict(list)

    mean_win_perc_dict['Fixed_StopLoss_TakeProfit_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_sp_testing']))
    mean_win_perc_dict['Fixed_Bar_testing'].\
        append(np.mean(entry_walk_forward_dict['fixed_bar_testing']))
    mean_win_perc_dict['Random_Exit_testing'].\
        append(np.mean(entry_walk_forward_dict['random_exit_testing']))
    mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    win_pc_df = pd.DataFrame(mean_win_perc_dict)

    return win_pc_df


## Exit testing

### Similar approach entry

In [7]:
@njit(cache=True)
def get_entry_exit_testing1(
    close_prices,
    open_prices,
    n_bars
):

    signal_list = np.zeros((2, len(close_prices)))

    # n_bars = np.int64(n_bars)

    for i in range(n_bars, len(signal_list[0])):
            
        if close_prices[i - n_bars] - close_prices[i - 1] > 0:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif close_prices[i - n_bars] - close_prices[i - 1] < 0:
           signal_list[0][i] = -1
           signal_list[1][i] = open_prices[i]
        else:
            pass
        
    return signal_list

@njit(cache=True)
def get_rsi(close_prices, prev_close_prices, length=14):
    # Create numpy arrays to store the gain/loss values
    gains = np.zeros(len(close_prices))
    losses = np.zeros(len(close_prices))

    # Iterate through the data frame and calculate the gain/loss for each period
    for i in range(1, len(close_prices)):
        change = close_prices[i] - prev_close_prices[i]
        if change > 0:
            gains[i] = change
        elif change < 0:
            losses[i] = abs(change)

    # Calculate the average gain and loss for each period
    avg_gains = np.zeros(len(close_prices))
    avg_losses = np.zeros(len(close_prices))
    for i in range(length, len(close_prices)):
        avg_gains[i] = np.mean(gains[i-length:i])
        avg_losses[i] = np.mean(losses[i-length:i])

    # Calculate the relative strength and RSI for each period
    rs = np.zeros(len(close_prices))
    rsi = np.zeros(len(close_prices))
    
    for i in range(len(close_prices)):
        if i+1 < length:
            rsi[i] = -999
        elif avg_losses[i] == 0:
            rs[i] = avg_gains[i]
            rsi[i] = 100
        else:
            rs[i] = avg_gains[i] / avg_losses[i]
            rsi[i] = 100 - (100 / (1 + rs[i]))

    return rsi

@njit(cache=True)
def get_entry_exit_testing2(
    close_prices, open_prices, prev_close_prices, rsi_window_size, rsi_threshold
):

    signal_list = np.zeros((2, len(close_prices)))

    rsi = get_rsi(close_prices, prev_close_prices, length=rsi_window_size)

    for i in range(len(close_prices)-1):

        if i < rsi_window_size - 1 or rsi[i] == -999:
            continue
       
        if rsi[i] < rsi_threshold:
            signal_list[0][i+1] = 1
            signal_list[1][i+1] = open_prices[i+1]
        elif rsi[i] > (100 - rsi_threshold):
            signal_list[0][i+1] = -1
            signal_list[1][i+1] = open_prices[i+1]
        else:
            pass

    return signal_list


### Random entry

In [8]:
@njit(cache=True)
def get_entry_exit_testing3(
    open_prices
):

    signal_list = np.zeros((2, len(open_prices)))

    for i in range(len(open_prices)-1):

        if np.random.rand() > 0.7:
            signal_list[0][i] = 1
            signal_list[1][i] = open_prices[i]
        elif np.random.rand() < 0.3:
            signal_list[0][i] = -1
            signal_list[1][i] = open_prices[i]
        else:
            pass

    return signal_list


### Winning percentage

In [9]:
def calculate_mean_win_perc_exit_testing(data, text_code):

    bars_per_5week = 7 * 60 * 24 * 5
    n_bars_per_year = 7 * 60 * 24 * 52

    n_not_worked = 0
    n_total_cases = 0

    exit_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = data.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        try:

            exec_dict = {'data': df}
            exec(text_code, exec_dict)
            df = exec_dict['df']

            commission = exec_dict['COMMISSION']
            slippage = exec_dict['SLIPPAGE'] 
            init_inv = exec_dict['AVAILABLE_CAPITAL']
            trade_size = exec_dict['TRADE_SIZE']       

            signal_idxs = exec_dict['buy_idxs'].copy()
            signal_idxs.extend(exec_dict['sell_idxs'])
            signal_idxs = sorted(signal_idxs)
            signal_idxs_true = [i - 1 for i in signal_idxs]

            df['exit_signal'] = 0
            df.loc[df.index.isin(signal_idxs_true[1:]), 'exit_signal'] = df.loc[df.index.isin(signal_idxs_true[1:]), 'signal'].values
            df['exit_prices'] = 0
            df.loc[df.index.isin(signal_idxs[1:]), 'exit_prices'] = df.loc[df.index.isin(signal_idxs[1:]), 'open'].values

            exit_list = np.zeros((2, df.shape[0]))
            exit_list[0][1:] = df['exit_signal'].values[:-1]
            exit_list[1] = df['exit_prices'].values

            # replacing entry with trend following entry
            signal_list = get_entry_exit_testing1(
                close_prices=df['close'].values,
                open_prices=df['open'].values,
                n_bars=5
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            fixed_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            exit_walk_forward_dict['trend_entry_testing'].append(fixed_winning_percent)

            # replacing entry with countertrend entry
            signal_list = get_entry_exit_testing2(
                close_prices=df['close'].values, 
                open_prices=df['open'].values, 
                prev_close_prices=df['close'].shift(1).fillna(method='bfill').values, 
                rsi_window_size=10, 
                rsi_threshold = 20
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            fixed_bar_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            exit_walk_forward_dict['countertrend_entry_testing'].append(fixed_bar_winning_percent)

            # random entry testing
            signal_list = get_entry_exit_testing3(
                open_prices=df['open'].values
            )

            signal_list, exit_list = get_signals(signal_list, exit_list)
            pos_open_prices, pos_exit_prices = create_position_open_prices(signal_list, exit_list)

            pnl_list = get_pnl_testing(
                trade_close_prices=pos_exit_prices,
                signal_list=exit_list[0], 
                trade_open_prices=pos_open_prices,
                commission=commission, 
                slippage=slippage, 
                init_inv=init_inv, 
                trade_size=trade_size
            )

            random_winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            exit_walk_forward_dict['random_entry_testing'].append(random_winning_percent)

        except:

            n_not_worked += 1

    mean_win_perc_dict = defaultdict(list)

    mean_win_perc_dict['Trend_testing'].\
        append(np.mean(exit_walk_forward_dict['trend_entry_testing']))
    mean_win_perc_dict['Countertrend_testing'].\
        append(np.mean(exit_walk_forward_dict['countertrend_entry_testing']))
    mean_win_perc_dict['Random_Entry_testing'].\
        append(np.mean(exit_walk_forward_dict['random_entry_testing']))
    mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    win_pc_df = pd.DataFrame(mean_win_perc_dict)

    return win_pc_df


## Core testing

### Winning percentage

In [10]:
def calculate_mean_win_perc_core_testing(data, text_code):

    bars_per_5week = 7 * 60 * 24 * 5
    n_bars_per_year = 7 * 60 * 24 * 52

    n_not_worked = 0
    n_total_cases = 0

    core_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = data.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        try:
            exec_dict = {'data': df}
            exec(text_code, exec_dict)
            df = exec_dict['df']
            pnl_list = exec_dict['all_arr']

            winning_percent = 100 * sum(pnl_list > 0) / np.sum(pnl_list != 0)
            core_walk_forward_dict['core_testing'].append(winning_percent)
        except:
            n_not_worked += 1

    mean_win_perc_dict = defaultdict(list)
    mean_win_perc_dict['Core_Testing'].\
        append(np.mean(core_walk_forward_dict['core_testing']))
    mean_win_perc_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    win_pc_df = pd.DataFrame(mean_win_perc_dict)

    return win_pc_df


### Performance

In [11]:
@njit(cache=True)
def get_drawdown(pnl_list):

    # running_total = np.cumsum(pnl_list)
    # max_running_total = np.maximum.accumulate(running_total)

    running_total = np.zeros(len(pnl_list))
    max_running_total = np.zeros(len(pnl_list))

    for i in range(len(pnl_list)):
        if i == 0:
            running_total[i] = pnl_list[i]
            max_running_total[i] = pnl_list[i]
        else:
            running_total[i] = running_total[i-1] + pnl_list[i]
            max_running_total[i] = max([max_running_total[i-1], pnl_list[i]])

    min_total = running_total[np.argmin(running_total)]

    max_total = max_running_total[np.argmin(running_total)]

    if max_total == 0:
        max_total = 0.00001

    max_drawdown = 100 * (min_total - max_total) / max_total

    if max_drawdown == -1:
        max_drawdown = 0

    return max_drawdown

@njit(cache=True)
def get_sharpe_ratio(
    pnl_list, 
    risk_free_rate=0
):
    # mean_return = np.mean(pnl_list)
    mean_return = 0
    for i in range(len(pnl_list)):
        mean_return += pnl_list[i]

    if len(pnl_list) == 0:
        mean_return = 0
    else:
        mean_return = mean_return / len(pnl_list)

    # std_dev = np.std(pnl_list)
    std_dev = 0
    for i in range(len(pnl_list)):
        std_dev += (pnl_list[i] - mean_return) ** 2

    if len(pnl_list) == 1:
        std_dev = np.sqrt(std_dev/len(pnl_list))
    else:
        std_dev = np.sqrt(std_dev/(len(pnl_list) - 1))

    if std_dev == 0:
        std_dev += 0.0001

    sharpe_ratio = (mean_return - risk_free_rate) / std_dev
    return sharpe_ratio


In [12]:
def calculate_mean_performance(data, text_code):

    bars_per_5week = 7 * 60 * 24 * 5
    n_bars_per_year = 7 * 60 * 24 * 52

    n_not_worked = 0
    n_total_cases = 0

    performance_walk_forward_dict = defaultdict(list)

    for idx in range(0, n_bars_per_year, bars_per_5week):

        n_total_cases += 1

        df = data.iloc[idx:idx+bars_per_5week, :]
        df.reset_index(drop=True, inplace=True)

        try:
            exec_dict = {'data': df}
            exec(text_code, exec_dict)
            df = exec_dict['df']
            pnl_list = exec_dict['all_arr']

            init_inv = exec_dict['AVAILABLE_CAPITAL']
            trade_size = exec_dict['TRADE_SIZE']       

            signal_idxs = exec_dict['buy_idxs'].copy()
            signal_idxs.extend(exec_dict['sell_idxs'])
            signal_idxs = sorted(signal_idxs)

            performance_walk_forward_dict['n_trades'].append(len(signal_idxs) - 1)
            performance_walk_forward_dict['pnl'].append(np.sum(pnl_list))
            performance_walk_forward_dict['roi'].append(100 * np.sum(pnl_list) / (trade_size * init_inv))
            performance_walk_forward_dict['avg_drawdown'].append(exec_dict['avg_drawdown'])
            performance_walk_forward_dict['drawdown'].append(get_drawdown(pnl_list))
            performance_walk_forward_dict['pnl_avgd_ratio'].append(exec_dict['fitness'])
            performance_walk_forward_dict['sharpe_ratio'].append(get_sharpe_ratio(pnl_list, risk_free_rate=0))
        except:
            n_not_worked += 1

    mean_perf_dict = defaultdict(list)
    mean_perf_dict['N_Trades'].\
        append(np.mean(performance_walk_forward_dict['n_trades']))
    mean_perf_dict['PNL'].\
        append(np.mean(performance_walk_forward_dict['pnl']))
    mean_perf_dict['ROI'].\
        append(np.mean(performance_walk_forward_dict['roi']))
    mean_perf_dict['AVG_Drawdown'].\
        append(np.mean(performance_walk_forward_dict['avg_drawdown']))
    mean_perf_dict['Drawdown'].\
        append(np.mean(performance_walk_forward_dict['drawdown']))
    mean_perf_dict['PNL_AVGD_Ratio'].\
        append(np.mean(performance_walk_forward_dict['pnl_avgd_ratio']))
    mean_perf_dict['Sharpe_Ratio'].\
        append(np.mean(performance_walk_forward_dict['sharpe_ratio']))
    mean_perf_dict['Not_Working'].\
        append(100 * n_not_worked / n_total_cases)
    
    perf_df = pd.DataFrame(mean_perf_dict)

    return perf_df


# Data loading function

In [13]:
def generate_52w_data(data_path):
    df = pd.read_csv(data_path)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.iloc[-(7 * 60 * 24 * 52):]
    df.sort_values('datetime', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df

# Testing strategies constructed from OLHCV lags only

In [14]:
data_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\datasets/\BTCUSD_ohlcv.csv')
df_52w = generate_52w_data(data_path)
df_52w.head()

,datetime,open,high,low,close,volume
0,2022-01-07 19:33:00,41733.5,41733.5,41695.0,41703.5,283302.0
1,2022-01-07 19:34:00,41703.5,41735.5,41703.5,41735.5,230678.0
2,2022-01-07 19:35:00,41735.5,41748.0,41735.0,41748.0,82516.0
3,2022-01-07 19:36:00,41751.5,41787.0,41751.5,41787.0,2612060.0
4,2022-01-07 19:37:00,41787.0,41787.0,41771.5,41771.5,100437.0


In [15]:
strategy_file_path = Path(r'C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\Upwork/\AlgoT_ML_Dev/\GrammarEvolution/\PonyGE2/\derived_strategies/\ge_results_ohlcv_lags2.csv')

df_str = pd.read_csv(strategy_file_path)
df_str = df_str[(df_str['fitness'] < 0) & (df_str['fitness'] < -2)]
df_str.sort_values('fitness', ascending=True, inplace=True)
df_str.reset_index(drop=True, inplace=True)
df_str

,buy,sell,fitness
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",-760.844894
1,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",-760.844894
2,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",-760.844894
3,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",-760.844894
4,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",-760.844894
...,...,...,...
3901,"df['volume'].values.reshape(-1, 1) > df['open_...","df['high'].values.reshape(-1, 1) == df['high_3...",-2.197323
3902,"df['close'].values.reshape(-1, 1) >= df['close...","df['volume_5'].values.reshape(-1, 1) <= df['hi...",-2.147685
3903,"df['volume_3'].values.reshape(-1, 1) / df['low...","df['volume'].values.reshape(-1, 1) / df['low_5...",-2.108416
3904,"df['high_3'].values.reshape(-1, 1) - df['open_...","np.log(df['low_5'].values.reshape(-1, 1)) + np...",-2.102275


In [16]:
final_entry_win_pc_df = pd.DataFrame()
final_exit_win_pc_df = pd.DataFrame()
final_core_win_pc_df = pd.DataFrame()
final_perf_df = pd.DataFrame()

lag_txt = '{i}'

for row in tqdm(df_str.iloc[:10].itertuples()):

    buy_signal_txt = row.buy
    sell_signal_txt = row.sell

    txt_code_ind = f'''import pandas as pd
import numpy as np
from src.fitness.indicators import indicators
from numba import njit
COMMISSION = 0.015
SLIPPAGE = 0.00005
AVAILABLE_CAPITAL = 700000
TRADE_SIZE = 0.5

@njit
def merge_pnl(arr1, arr2):
    out = np.zeros((len(arr1) + len(arr2)))
    idx = 1
    for i in range(len(arr1) + len(arr2)):
        if i % 2 == 0:
            out[i] = arr1[int(i/2)]
        else:
            out[i] = arr2[i-idx]
        idx += 1
    return out

@njit
def get_drawdowns(arr):
    drawdowns = np.zeros((len(arr)))
    max = arr[0]
    for i in range(1, len(drawdowns)-1):
        if arr[i-1] > arr[i] and arr[i] < arr[i+1]:
            min = arr[i]
            drawdowns[i] = max - min
        elif arr[i-1] < arr[i] and arr[i] > arr[i+1]:
            max = arr[i]
    return drawdowns

@njit
def get_pnl(trade_close_prices, trade_open_prices, commission, slippage, init_inv, trade_size, is_buy):
    pnl_list = np.zeros(len(trade_close_prices))
    for i in range(len(trade_close_prices)):
        temp_n_assets = int(init_inv * trade_size / trade_open_prices[i])
        if is_buy == 1:
            temp_pnl = temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 + slippage))
        else:
            temp_pnl = -temp_n_assets * (trade_close_prices[i] - trade_open_prices[i] * (1 - slippage))
        temp_pnl = temp_pnl * (1 - commission)
        init_inv += temp_pnl
        pnl_list[i] = temp_pnl
    return pnl_list

df = data.copy()
for i in range(1, 6):
    df[f'close_{lag_txt}'] = df['close'].shift(i)
    df[f'open_{lag_txt}'] = df['open'].shift(i)
    df[f'high_{lag_txt}'] = df['high'].shift(i)
    df[f'low_{lag_txt}'] = df['low'].shift(i)
    df[f'volume_{lag_txt}'] = df['volume'].shift(i)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df['buy'] = ({buy_signal_txt}).astype(int)
df['sell'] = ({sell_signal_txt}).astype(int)
df['signal'] = df['buy'] + df['sell']
df['signal'] = df['signal'].apply(lambda x: 1 if x==1 else 0)
df['sell'] = df['sell'] * (-1)
df['signal'] = df['signal'] * df['sell']
df['signal'] = df['signal'] + df['buy']
df.drop(columns=['buy', 'sell'], inplace=True)

buy_idxs = []
sell_idxs = []
is_buy = 0
is_sell = 0
for i, row in enumerate(df.itertuples()):
    if row.signal == 1 and is_buy == 0:
        buy_idxs.append(i+1)
        is_buy = 1
        is_sell = 0
    elif row.signal == -1 and is_sell == 0:
        sell_idxs.append(i+1)
        is_sell = 1
        is_buy = 0

if len(buy_idxs) > len(sell_idxs):
    buy_idxs = buy_idxs[:-(len(buy_idxs) - len(sell_idxs))]
elif len(buy_idxs) < len(sell_idxs):
    sell_idxs = sell_idxs[:-(len(sell_idxs) - len(buy_idxs))]

if len(buy_idxs) == 0 or len(sell_idxs) == 0:
    print("not enough signals")

buy_prices = df[df.index.isin(buy_idxs)]['open'].values
sell_prices = df[df.index.isin(sell_idxs)]['open'].values

if buy_idxs[0] < sell_idxs[0]:
    buy_arr = get_pnl(sell_prices, buy_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
    buy_pnl = np.sum(buy_arr)
    sell_arr = get_pnl(buy_prices[1:], sell_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
    sell_pnl = np.sum(sell_arr)
    all_arr = merge_pnl(buy_arr, sell_arr)
else:
    sell_arr = get_pnl(buy_prices, sell_prices, COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 0)
    sell_pnl = np.sum(sell_arr)
    buy_arr = get_pnl(sell_prices[1:], buy_prices[:-1], COMMISSION, SLIPPAGE, AVAILABLE_CAPITAL, TRADE_SIZE, 1)
    buy_pnl = np.sum(buy_arr)
    all_arr = merge_pnl(sell_arr, buy_arr)

total_pnl = buy_pnl + sell_pnl
equity_curve_arr = np.cumsum(all_arr)
drawdowns = get_drawdowns(equity_curve_arr)
avg_drawdown = np.sum(drawdowns[drawdowns!=0]) / len(drawdowns[drawdowns!=0])
fitness = total_pnl / avg_drawdown'''

    entry_win_pc_df = calculate_mean_win_perc_entry_testing(
        data=df_52w, 
        text_code=txt_code_ind
    )

    temp_signal_df = pd.DataFrame({'buy': [buy_signal_txt], 'sell': [sell_signal_txt]})
    entry_win_pc_df = pd.concat([temp_signal_df, entry_win_pc_df], axis=1)
    final_entry_win_pc_df = pd.concat([final_entry_win_pc_df, entry_win_pc_df])

    exit_win_pc_df = calculate_mean_win_perc_exit_testing(
        data=df_52w, 
        text_code=txt_code_ind
    )

    exit_win_pc_df = pd.concat([temp_signal_df, exit_win_pc_df], axis=1)
    final_exit_win_pc_df = pd.concat([final_exit_win_pc_df, exit_win_pc_df])

    core_win_pc_df = calculate_mean_win_perc_core_testing(
        data=df_52w, 
        text_code=txt_code_ind
    )

    core_win_pc_df = pd.concat([temp_signal_df, core_win_pc_df], axis=1)
    final_core_win_pc_df = pd.concat([final_core_win_pc_df, core_win_pc_df])

    perf_df = calculate_mean_performance(
        data=df_52w, 
        text_code=txt_code_ind
    )

    perf_df = pd.concat([temp_signal_df, perf_df], axis=1)
    final_perf_df = pd.concat([final_perf_df, perf_df])

10it [04:46, 28.67s/it]


In [17]:
final_entry_win_pc_df

,buy,sell,Fixed_StopLoss_TakeProfit_testing,Fixed_Bar_testing,Random_Exit_testing,Not_Working
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,44.233134,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,45.248965,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,44.615141,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.860668,50.201439,43.977003,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,45.042902,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,44.906170,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,46.132500,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,47.093857,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,43.255824,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",44.451349,46.107231,44.435770,0.0


In [18]:
final_exit_win_pc_df

,buy,sell,Trend_testing,Countertrend_testing,Random_Entry_testing,Not_Working
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,44.205838,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,44.537117,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,44.369603,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.354151,58.393833,44.327389,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,44.025109,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,44.644633,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,44.309340,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,44.191204,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,43.950200,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",56.291363,58.383999,44.510973,0.0


In [19]:
final_core_win_pc_df

,buy,sell,Core_Testing,Not_Working
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",33.617697,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",29.569133,0.0


In [20]:
final_perf_df

,buy,sell,N_Trades,PNL,ROI,AVG_Drawdown,Drawdown,PNL_AVGD_Ratio,Sharpe_Ratio,Not_Working
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",2998.636364,-107194.746293,-30.627070,1349.171210,-1472.405234,-58.505676,0.046052,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0
0,"df['volume_2'].values.reshape(-1, 1) > df['ope...","df['high_3'].values.reshape(-1, 1) - df['close...",3117.727273,-143079.948712,-40.879985,1253.440651,-1603.559323,-53.955291,-0.043887,0.0


In [21]:
final_entry_win_pc_df.to_csv('testing_results/entry_testing_lag.csv', index=False)
final_exit_win_pc_df.to_csv('testing_results/exit_testing_lag.csv', index=False)
final_core_win_pc_df.to_csv('testing_results/core_testing_lag.csv', index=False)
final_perf_df.to_csv('testing_results/perf_lag.csv', index=False)